In [ ]:
import pyreadstat
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import sklearn
from sklearn import tree
from sklearn import metrics
from sklearn.manifold import TSNE
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import ParameterGrid

In [ ]:
hip_fracture_df, hip_fracture_metadata = pyreadstat.read_sav(r'C:\Users\Asus\Downloads/HipFractureQualityData.sav')
print(dir(hip_fracture_metadata))


In [ ]:
column_labels = hip_fracture_metadata.column_labels 
column_mapping = {column: column_labels[i] for i, column in enumerate(hip_fracture_df.columns)}
print(column_mapping)

In [ ]:
hip_fracture_df = pd.DataFrame(hip_fracture_df)
new_df = pd.DataFrame(hip_fracture_df)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
hip_fracture_df.info()
hip_fracture_df.head()
hip_fracture_df['RACE_NEW'].value_counts()



In [ ]:
hip_fracture_df['RACE_NEW'].value_counts()

In [ ]:
hip_fracture_df.head()

In [ ]:
hip_fracture_df.shape

In [ ]:
#  age (years), gender (female/
# male), body mass index (BMI) (kg/m2
# ), fracture type
# (femoral neck Garden types 1 and 2—non-displaced/
# femoral neck Garden type 3 and 4-displaced/intertrochanteric/
# subtrochanteric), procedure (ORIF femoral neck, ORIF
# inter/subtrochanteric with plate/screw implant, ORIF inter/
# subtrochanteric with intramedullary implant), American
# Society of Anesthesiologist (ASA) class (I/II/III/IV),
# functional status (independent defined as patients do not
# require assistance from another person for any activities of
# daily living/partially depended defined as patients requires
# some assistance from another person for activities of daily

In [ ]:
# Age (years)
# 60+ 3151 (11.2)
# 70+ 6247 (22.1)
# 80+ 11 691 (41.4)
# 90+ 7118 (25.2)
# Female sex 19 845 (70.4)
# Body mass index (kg/m2
# ) 24.3 (21.6–27.3)
# Fracture type
# Femoral neck fracture (sub capital, Garden types 1 and 2)-nondisplaced 2479 (8.8)
# Femoral neck fracture (sub capital, Garden types 3 and 4)-displaced 8324 (29.5)
# Intertrochanteric 15 761 (55.9)
# Subtrochanteric 1642 (5.8)
# ASA classification
# I 126 (.4)
# II 4162 (14.8)
# III 17 631 (62.5)
# IV 6288 (22.3)
# Functional status
# Independent 21 672 (76.8)
# Partially dependent 5651 (20.0)
# Totally dependent 884 (3.1)
# Preoperative dementia 8668 (30.7)
# Preoperative delirium 3714 (13.2)
# Preoperative bone protective medication prescription 9047 (32.1)
# Preoperative need for mobility aid 16 239 (57.6)
# Preoperative pressure sore 971 (3.4)
# Medical co-management
# No 3071 (10.9)
# Yes-co-management throughout stay 20 576 (72.9)
# Yes-partial co-management during stay 4560 (16.2)
# Standardized hip fracture protocol 15 808 (56.0)
# Diabetes
# Insulin dependent 2026 (7.2)
# Non-insulin dependent 3035 (10.8)
# No 23 146 (82.1)
# Smoking 2848 (10.1)
# Dyspnea
# At rest 273 (1.0)
# Moderate exertion 1880 (6.7)
# No 26 054 (92.4)
# Chronic obstructive pulmonary disorder 3035 (10.8)
# Congestive heart failure 1051 (3.7)
# Hypertension requiring medication 19 120 (67.8)
# Acute renal failure 150 (.5)
# Dialysis 522 (1.9)
# Disseminated cancer 407 (1.4)
# Wound infection 1095 (3.9)
# Preoperative steroid use 1476 (5.2)
# Weight loss >10% body weight in last 6 months 384 (1.4)
# Bleeding disorder 4796 (17.0)
# Transfusion 1205 (4.3)
# (continued)
# 4 Geriatric Orthopaedic Surgery & Rehabilitation 12(0)
# Results
# In total, 28 207 patients underwent surgery for a hip
# fracture of which 8030 (28.5%) developed a delirium
# within 30 days after surgery (Table 1). The majority of
# patients was female (n = 19 845, 70.4%) and aged 80 years
# or older (n = 18 809, 66.6%).
# Variables Selection
# Variables selection identified the following predictive
# variables for prediction of postoperative delirium: (1) age,
# (2) ASA class, (3) functional status, (4) preoperative dementia, (5) preoperative delirium, and (6) preoperative
# need for mobility-aid (Figure 1).
# Model Performance
# The model performance of the conducted algorithms varied
# as measured by c-statistic from .67 to .79. The calibration
# slopes ranged from .44 to 1.02, and the calibration intercepts
# ranged from .01 to 1.06. The Null model Brier score was
# .20, calculated based on an incidence of 28.5%. The Brier
# scores ranged from .15 to .18 (Table 2).
# Although model performance metrics were comparable,
# except the RF algorithm, the PLR model had the most
# clinically meaningful variable importance and was
# therefore chosen as the best algorithm and therefore the
# final model (Figure 1).
# Decision curve analysis of the PLR model revealed that
# decision changes based on the model outperformed not
# only the default strategies of decision change looking at all
# patients or no patients, but also that the decision changes
# based on presence of only a preoperative delirium alone
# (Figure 2).
# Available Web-Application
# The PLR algorithm was incorporated into a web-based
# application and deployed as an open-access available tool
# for clinicians: https://sorg-apps.shinyapps.io/hipfxdelirium/
# Table 1. (continued)
# Variable n (%) | Median (IQR)
# Systemic inflammatory response syndrome (SIRS)
# None 25 478 (90.3)
# SIRS 2566 (9.1)
# Sepsis 163 (.6)
# Sodium (mg/dL) 138.0 (136.0–140.0)
# Creatinine (mg/dL) .88 (.70–1.13)
# White blood cell (×103
# /µL) 9.60 (7.60–11.90)
# Hematocrit (%) 35.0 (31.0–38.6)
# Platelet (×103
# /µL)

In [ ]:
#(1) age,
# (2) ASA class, (3) functional status, (4) preoperative dementia, (5) preoperative delirium, and (6) preoperative
# need for mobility-aid (Figure 1)

In [ ]:
readd_col = pd.DataFrame(new_df["ReadmissionPost_117"])

In [ ]:
readd_col.head()

In [ ]:
new_df.drop(new_df.iloc[:, 0:225], inplace=True, axis=1)

In [ ]:
new_df.head()

In [ ]:
new_df.drop(['BMICategories','ReadmissionPost_117','ReadmissionPost_119','Readmission2Post_1','Readmission2Post_2','Readmission2Post_3','Readmission3Post_4','Readmission3Post_5'], axis=1,inplace=True)

In [ ]:
new_df.head()

In [ ]:
new_df["ReadmissionPost_117"] = readd_col

In [ ]:
new_df.head(20)

In [ ]:
new_df.shape

In [ ]:
new_df.isnull().values.any()

In [ ]:
new_df.isnull().sum().sum()

In [ ]:
new_df.dropna(inplace=True)

In [ ]:
new_df.shape

In [ ]:
new_df.head(20)

In [ ]:
new_df.drop(['PropensityScore22','casematch22','BMI'], axis=1,inplace=True)

In [ ]:
new_df.head(20)

In [ ]:
X = hip_fracture_df.iloc[:,[1,2,4,14,15,25,34,35,36,37,110]]
print(X)
# y = hip_fracture_df.iloc[:,[13]]
# print(y)


In [ ]:
y = hip_fracture_df.iloc[:,[198]]
print(y)


In [ ]:
from sklearn import preprocessing
#,'HIP_PREOP_DELIRIUM','HIP_PREOP_MOBAID', 'FNSTATUS2', 'ASACLAS'
label_encoder = preprocessing.LabelEncoder()

X.apply(label_encoder.fit_transform)

In [ ]:
y['READMISSION1'] = label_encoder.fit_transform(y['READMISSION1'])
print(y['READMISSION1'].unique())

In [ ]:
from sklearn import preprocessing
#,'HIP_PREOP_DELIRIUM','HIP_PREOP_MOBAID', 'FNSTATUS2', 'ASACLAS'
#HIP_PREOP_DEMENTIA	HIP_PREOP_DELIRIUM	HIP_PREOP_BONEMEDS	HIP_PREOP_MOBAID	HIP_PREOP_PRESSORE
#HIP_POST_PRESSORE	HIP_POST_DELIRIUM	HIP_POST_MOBAID
#RACE_NEW
#DIABETES	SMOKE	DYSPNEA	
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.
# X['HIP_PREOP_DEMENTIA']= label_encoder.fit_transform(X['HIP_PREOP_DEMENTIA'])
# X['HIP_PREOP_DELIRIUM']= label_encoder.fit_transform(X['HIP_PREOP_DELIRIUM'])
# X['HIP_PREOP_MOBAID']= label_encoder.fit_transform(X['HIP_PREOP_MOBAID'])
# X['FNSTATUS2']= label_encoder.fit_transform(X['FNSTATUS2'])
# X['ASACLAS']= label_encoder.fit_transform(X['ASACLAS'])
# X['AGE']= label_encoder.fit_transform(X['AGE'])
# X['HIP_POST_PRESSORE']= label_encoder.fit_transform(X['HIP_POST_PRESSORE'])
# X['HIP_POST_DELIRIUM']= label_encoder.fit_transform(X['HIP_POST_DELIRIUM'])
# X['HIP_POST_MOBAID']= label_encoder.fit_transform(X['HIP_POST_MOBAID'])
# X['RACE_NEW']= label_encoder.fit_transform(X['RACE_NEW'])
# X['DIABETES']= label_encoder.fit_transform(X['DIABETES'])
# X['SMOKE']= label_encoder.fit_transform(X['SMOKE'])
# X['DYSPNEA']= label_encoder.fit_transform(X['DYSPNEA'])
  
# print(X['HIP_PREOP_DEMENTIA'].unique())
# print(X['HIP_PREOP_DELIRIUM'].unique())
# print(X['HIP_PREOP_MOBAID'].unique())
# print(X['FNSTATUS2'].unique())
# print(X['ASACLAS'].unique())
# print(X['AGE'].unique())
# print(X['HIP_POST_PRESSORE'].unique())
# print(X['HIP_POST_DELIRIUM'].unique())
# print(X['HIP_POST_MOBAID'].unique())
# print(X['RACE_NEW'].unique())
# print(X['DIABETES'].unique())
# print(X['SMOKE'].unique())
# print(X['DYSPNEA'].unique())

# y['HIP_POST_DELIRIUM'] = label_encoder.fit_transform(y['HIP_POST_DELIRIUM'])
# print(y['HIP_POST_DELIRIUM'].unique())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
# Create a Random Forest Classifier
rf = RandomForestClassifier(criterion='gini', max_depth=5, n_estimators=20, random_state=0)

# Train the model using the training sets
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
# Please write your code below:

param = ParameterGrid({'n_estimators': np.arange(50, 201, 50),
         'criterion': ['gini', 'entropy'],
         'max_depth': np.arange(5, 21, 5)
         }
)
# rfx = GridSearchCV(RandomForestClassifier(random_state=0), param)
# rfx = rfx.fit(X_train,y_train)

acc = 0
for para in param:
  n_es,cri, maxd = para['n_estimators'],para['criterion'],para['max_depth']
  rfx = RandomForestClassifier(criterion=cri, max_depth=maxd, n_estimators=n_es, random_state=0)
  rfx = rfx.fit(X_train,y_train)
  y_pred_rfx = rfx.predict(X_test)
  accu = metrics.accuracy_score(y_test,y_pred_rfx)
  if accu > acc:
    acc = accu
    par1 = para

print(para)
print("Accuracy",acc)

In [ ]:
# get importance
rf_importance = rfx.feature_importances_

# summarize and print out the feature importance
non_zero_index_list_rf = [i for i, v in enumerate(rf_importance) if v != 0]

# remove all zero value entries
non_zero_feature_names_rf = [list(X.columns)[i] for i in non_zero_index_list_rf]
non_zero_importance_rf = [rf_importance[i] for i in non_zero_index_list_rf]

# draw graph
fig,ax = plt.subplots(figsize=(50,50))
plt.barh(non_zero_feature_names_rf, non_zero_importance_rf)
ax.set_yticklabels(non_zero_feature_names_rf, fontsize=12)
plt.xlabel('Entropy Importance')
plt.ylabel('Features (with non-zero entropy importance)')
plt.show()